In [ ]:
import duckdb
import pandas as pd

conn = duckdb.connect()

print("Filtering English songs directly from parquet...")

lyrics = conn.execute("""
    SELECT *
    FROM read_parquet('../data/song_lyrics.parquet')
    WHERE language = 'en'
    ORDER BY views DESC
    LIMIT 5000
""").fetchdf()

print(f"Loaded only English top 5000 rows: {len(lyrics)}")

metadata = conn.execute("""
    SELECT *,
           lower(track_name) AS track_name_clean,
           lower(artists) AS artists_clean
    FROM read_parquet('../data/song_metadata.parquet')
""").fetchdf()

print(f" Metadata loaded: {len(metadata)} rows")

def clean(x):
    return str(x).lower().strip() if pd.notnull(x) else ""

lyrics["join_key"] = lyrics["title"].apply(clean) + " " + lyrics["artist"].apply(clean)
metadata["join_key"] = metadata["track_name_clean"] + " " + metadata["artists_clean"]

metadata = metadata.drop_duplicates(subset="join_key", keep="first")

merged = lyrics.merge(metadata, on="join_key", how="inner")

print(f"Matched: {len(merged)} rows")
print(f"Match rate: {len(merged)/len(lyrics)*100:.2f}%")

merged.to_parquet("../data/final_joined_english.parquet", index=False)

print("Saved final_joined_english.parquet")


Filtering English songs directly from parquet...
Loaded only English top 1000 rows: 5000
 Metadata loaded: 114000 rows
Matched: 632 rows
Match rate: 12.64%
Saved final_joined_english.parquet


In [7]:
df = pd.read_parquet("../data/final_joined_english.parquet")
print("Loaded dataset:", df.shape)

Loaded dataset: (632, 35)
